# Stream the raw measurements and plot them

In [ ]:
import serial
import struct, time
import numpy as np
serial_port = '/dev/ttyACM0'
ADC_BUFFER_LENGTH = 4048
ADC_BUFFER_LENGTH = 1
DATA_DIR = 'data'

In [ ]:
def cal_fft(values):
    values = np.array(values, dtype = np.float)
    values = (values - 32767) * (3.3 / 65535.0)
    return np.absolute(np.fft.rfft(values))[1:]

def get_freqs():
    sampling_frequency = 1000
    signal_length = ADC_BUFFER_LENGTH
    return np.arange(ADC_BUFFER_LENGTH/2) * sampling_frequency / signal_length

In [ ]:
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.models.tools import HoverTool
from bokeh.io import output_notebook, show, push_notebook
from bokeh.server.server import Server
from bokeh.application import Application
from tornado.ioloop import IOLoop
from bokeh.application.handlers.function import FunctionHandler

# output_notebook()
# output_file("lines.html")

In [ ]:
from threading import Thread, Lock

class Recording:
    def __init__(self, serial_dev:str = serial_port,recording_name:str = ''):
        self._restart_buffers()
        self.name = recording_name
        self.plotter = None
        self.logging_thread = None
        self.stop_threads = False
        self.serial_dev = serial_dev
        self.new_value_lock = Lock()
    
    def _restart_buffers(self):
        self.values = np.array([], dtype=np.int64).reshape(0)
        self.indexes = np.array([], dtype=np.int64).reshape(0)
        self._restart_new_data_buffer()
    
    def _restart_new_data_buffer(self):
        self.new_values = np.array([], dtype=np.int64).reshape(0)
        self.new_indexes = np.array([], dtype=np.int64).reshape(0)
        
    def _main_logging_function(self):
        self._restart_buffers()
        period = .001  # in seconds (simulate waiting for new data)
        last_timestamp = 0
        num_reads = 25

        with serial.Serial(serial_port, 115200) as ser:
            while True:
                byts = ser.read(num_reads * 2)

                if self.stop_threads:
                    break
                    
                values = struct.unpack('{}H'.format(num_reads), byts)
                timestamp = np.linspace(last_timestamp, last_timestamp + (period*num_reads), num_reads, endpoint=False)
                last_timestamp = timestamp[-1]+period
                
                with self.new_value_lock:
                    self.new_values = np.hstack([self.new_values, values])
                    self.new_indexes = np.hstack([self.new_indexes, timestamp])
                
                self.values = np.hstack([self.values, values])
                self.indexes = np.hstack([self.indexes, timestamp])
                
                # fft = cal_fft(values)
                # source.data = dict(
                #     time = freqs,
                #     value = fft
                # )
                
                if self.plotter != None:
                    self.plotter.update_plot(timestamp, values)
                    
    
    def get_new_data(self):
        with self.new_value_lock:
            to_return = (np.copy(self.new_values), np.copy(self.new_indexes))
            self._restart_new_data_buffer()
        return to_return
    
    def _flush_input(self):
        # clean serial port input buffer (ignore previous data)
        with serial.Serial(serial_port, 115200) as ser:
            ser.flushInput()
    
    def start_logging(self):
        self._send_single_byte(0)
        if self.logging_thread == None:
            self.logging_thread = Thread(target=self._main_logging_function)
        if self.plotter:
            self.plotter.init()
        self.stop_threads = False
        self._flush_input()
        self._send_single_byte(1)
        self.logging_thread.start()
        print("logging started")
            
    def stop_logging(self):
        self.stop_threads = True
        self.logging_thread.join()
        self._send_single_byte(0)
        self.logging_thread = None
        self.save_data()
        print("logging stopped")
    
    def _send_single_byte(self, byt: int):
        with serial.Serial(serial_port, 115200) as ser:
            ser.write(bytes([byt]))
    
    def save_data(self):
        timestr = time.strftime("%Y%m%d-%H%M%S")
        with open(f'{DATA_DIR}/{self.name}{timestr}.npy','wb') as f:
            np.save(f, self.values)
            np.save(f, self.indexes)
    
    
class LivePlotter:
    def __init__(self, n_show = 25* 1000):
        self.n_samples_to_show = n_show
    
    def update_plot(self,time,value):
        new_data=dict(time=[], value=[])
        new_data['time'] = time
        new_data['value'] = value
        self.data_source.stream(new_data, self.n_samples_to_show)
        push_notebook(handle=self.handle)
    
    def init(self):
        hTool = HoverTool(
            tooltips = [
                ("frequency", "@time"),
                ("value", "@value"),
            ],
            mode='vline'
        )
        
        self.figure = figure(plot_width=900, plot_height=500)
        self.figure.y_range=Range1d(0, (2**16)-1)
        self.figure.add_tools(hTool)
        self.data_source = ColumnDataSource(data=dict(time=[], value=[]))
        self.line = self.figure.line(x='time' , 
                                     y='value', 
                                     source = self.data_source, 
                                     legend_label="ADC", 
                                     line_width=2)
        self.handle = show(self.figure, notebook_handle=True)
#         save(self.figure)
        

recording = Recording()
# recording.plotter = LivePlotter()

recording.start_logging()
s = time.time()

recording.stop_logging()
print(time.time() - s)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

class BokehApp(Recording):
    def __init__(self, recorder):
        io_loop = IOLoop.current()
        self.recorder = recorder
        self.recorder.start_logging()
        server = Server(applications = {'/myapp': Application(FunctionHandler(self.make_document))}, 
                        io_loop = io_loop,
                        port = 9000)
        server.start()
#         server.show('/myapp')
        io_loop.start()


    def make_document(self, doc):
        hTool = HoverTool(
            tooltips = [
                ("frequency", "@time"),
                ("value", "@value"),
            ],
            mode='vline'
        )
        
        fig = figure(plot_width=1500, plot_height=500)
        fig.y_range=Range1d(0, (2**16)-1)
        fig.add_tools(hTool)
        self.data_source = ColumnDataSource(data=dict(time=[], value=[]))
        line = fig.line(x='time' , 
                                     y='value', 
                                     source = self.data_source, 
                                     legend_label="ADC", 
                                     line_width=2)

        def update():
            vals, idxs = self.recorder.get_new_data()
            if vals.shape[0] == 0:
                return
            new_data=dict(time=[], value=[])
            new_data['time'] = idxs
            new_data['value'] = vals
            try:
                self.data_source.stream(new_data, 1000*20)
            except Exception as e:
                print(e)
                print(idxs.shape)
                print(vals.shape)
                

        doc.add_root(fig)
        doc.add_periodic_callback(update, 50)

In [ ]:
app = BokehApp(recording)